In [1]:
import sys
sys.path.append('..')

# Imports



In [21]:
from uuid import uuid4
from tempfile import gettempdir
from os.path import join
from aequilibrae import Project
import shapely.wkb
from shapely.geometry import Point
from math import sqrt
import pandas as pd
import numpy as np
from time import perf_counter
from copy import deepcopy
from aequilibrae.paths import NetworkSkimming

We create an empty project on an arbitrary folder



In [3]:
fldr = 'D:/release/Sample models/Chicago_2020_02_15/1_project'
fldr = 'D:/GDrive/DATA/Pedro/Professional/Consulting/University of Arkansas/2020-09 - Network criticality analysis/DATA/model'
project = Project()
project.open(fldr)

Now we can download the network from any place in the world (as long as you have memory for all the download
and data wrangling that will be done)



In [4]:
network = project.network
network.build_graphs(modes='c')
orig_graph = network.graphs['c']
orig_nodes_to_indices = orig_graph.nodes_to_indices
net = orig_graph.network
orig_centroids = orig_graph.centroids

In [10]:
graph = pd.DataFrame(orig_graph.graph, copy=True)
centroids = np.arange(orig_centroids.shape[0])
t = perf_counter()

# Build link index
link_idx = np.empty(net.link_id.max() + 1).astype(np.int)
link_idx[net.link_id] = np.arange(net.shape[0])

nodes = np.hstack([net.a_node.values, net.b_node.values])
links = np.hstack([net.link_id.values, net.link_id.values])

idx = np.argsort(nodes)
all_nodes = nodes[idx]
all_links = links[idx]
links_index = np.empty(all_nodes.max() + 1, np.int64)
links_index.fill(-1)
nlist = np.arange(all_nodes.max())

y, x, _ = np.intersect1d(all_nodes, nlist, assume_unique=False, return_indices=True)
links_index[y] = x[:]
links_index[-1] = all_links.shape[0]
for i in range(all_nodes.max(), 1, -1):
    links_index[i - 1] = links_index[i] if links_index[i - 1] == -1 else links_index[i - 1]

nodes = np.hstack([net.a_node.values, net.b_node.values])
counts = np.bincount(nodes)
counts[centroids] = 999

truth = (counts == 2).astype(np.int)
link_edge = truth[net.a_node.values] + truth[net.b_node.values]
link_edge = net.link_id.values[link_edge == 1]

simplified_links = np.empty(net.link_id.max() + 1)
simplified_links.fill(-1)
simplified_links = simplified_links.astype(np.int)
simplified_directions = np.zeros(net.link_id.max() + 1, np.int)

compressed_dir = np.zeros(net.link_id.max() + 1, np.int)
compressed_a_node = np.zeros(net.link_id.max() + 1, np.int)
compressed_b_node = np.zeros(net.link_id.max() + 1, np.int)

slink = 0
major_nodes = {}
tot = 0
tot_graph_add = 0
for pre_link in link_edge:
    if simplified_links[pre_link] >= 0:
        continue
    ab_dir = 1
    ba_dir = 1
    lidx = link_idx[pre_link]
    a_node = net.a_node.values[lidx]
    b_node = net.b_node.values[lidx]
    direction = net.direction.values[lidx]
    n = a_node if counts[a_node] == 2 else b_node
    first_node = b_node if counts[a_node] == 2 else a_node

    ab_dir = 0 if (first_node == a_node and direction < 0) or (first_node == b_node and direction > 0) else ab_dir
    ba_dir = 0 if (first_node == a_node and direction > 0) or (first_node == b_node and direction < 0) else ba_dir

    while counts[n] == 2:
        if simplified_links[pre_link] >= 0:
            raise Exception('How the heck did this happen?')
        simplified_links[pre_link] = slink
        simplified_directions[pre_link] = 1 if a_node == n else -1
        for k in range(links_index[n], links_index[n + 1]):
            lnk = all_links[k]
            if lnk == pre_link:
                continue
            break
        if lnk == pre_link:
            raise Exception('How the heck did this happen again?')
        pre_link = lnk
        lidx = link_idx[pre_link]
        a_node = net.a_node.values[lidx]
        b_node = net.b_node.values[lidx]
        direction = net.direction.values[lidx]
        ab_dir = 0 if (n == a_node and direction < 0) or (n == b_node and direction > 0) else ab_dir
        ba_dir = 0 if (n == a_node and direction > 0) or (n == b_node and direction < 0) else ba_dir
        n = net.a_node.values[lidx] if n == net.b_node.values[lidx] else net.b_node.values[lidx]
        direc = 0

    if max(ab_dir, ba_dir) < 1:
        tot += 1

    tot_graph_add += ab_dir + ba_dir
    simplified_links[pre_link] = slink
    simplified_directions[pre_link] = 1 if a_node == n else -1
    last_node = b_node if counts[a_node] == 2 else a_node
    major_nodes[slink] = [first_node, last_node]

    # Available directions are NOT indexed like the other arrays
    compressed_a_node[slink] = first_node
    compressed_b_node[slink] = last_node
    if ab_dir > 0:
        if ba_dir > 0:
            compressed_dir[slink] = 0
        else:
            compressed_dir[slink] = 1
    elif ba_dir > 0:
        compressed_dir[slink] = -1
    else:
        compressed_dir[slink] = -999
    slink += 1

print('Network simplification', perf_counter() - t)

t = perf_counter()
links_to_remove = np.argwhere(simplified_links >= 0)
new_graph = pd.DataFrame(graph, copy=True)
new_graph = new_graph[~new_graph.link_id.isin(links_to_remove[:, 0])]

indices = pd.DataFrame(graph[['id', 'link_id', 'direction']], copy=True)
indices.loc[:, 'link_id'] = graph.link_id * graph.direction
indices.set_index('link_id', inplace=True)

comp_lnk = pd.DataFrame({'a_node': compressed_a_node[:slink],
                         'b_node': compressed_b_node[:slink],
                         'direction': compressed_dir[:slink],
                         '__augmented_link_id__': np.arange(slink),
                         'distance':np.zeros(slink, dtype=np.float64)})
comp_lnk.loc[:, 'a_node'] = orig_nodes_to_indices[comp_lnk.a_node]
comp_lnk.loc[:, 'b_node'] = orig_nodes_to_indices[comp_lnk.b_node]

comp_lnk_ab = pd.DataFrame(comp_lnk[comp_lnk.direction >= 0])
comp_lnk_ab.loc[:, 'direction'] = 1

comp_lnk_ba = pd.DataFrame(comp_lnk[comp_lnk.direction <= 0])
comp_lnk_ba.loc[:, "direction"] = -1
aux = np.array(comp_lnk_ba.a_node.values, copy=True)
comp_lnk_ba.loc[:, "a_node"] = comp_lnk_ba.loc[:, "b_node"]
comp_lnk_ba.loc[:, "b_node"] = aux[:]

df = pd.concat([new_graph, comp_lnk_ab, comp_lnk_ba])
df = df.assign(__new_id__=0)

# centroids will not change their IDs, as they are kept the same (the first N nodes in the graph)
nodes = np.unique(np.hstack((df.a_node.values, df.b_node.values))).astype(np.int64)
nodes = np.setdiff1d(nodes, centroids, assume_unique=True)
compress_nodes = np.hstack((centroids, nodes)).astype(np.int64)
compress_num_nodes = compress_nodes.shape[0]
# nodes_to_indices = np.empty(int(compress_nodes.max()) + 1, np.int64)
# nodes_to_indices.fill(-1)
nlist = np.arange(compress_num_nodes)
# nodes_to_indices[compress_nodes] = nlist
num_links = df.shape[0]
# df = df.sort_values(by=["a_node", "b_node"])
# df.loc[:, "a_node"] = nodes_to_indices[df.a_node.values][:]
# df.loc[:, "b_node"] = nodes_to_indices[df.b_node.values][:]
df = df.sort_values(by=["a_node", "b_node"])
df.index = np.arange(df.shape[0])
df.loc[:, "__new_id__"] = np.arange(df.shape[0])
df.loc[:, 'id'] = df.__new_id__[:]
compressed_fs = np.empty(compress_num_nodes + 1, dtype=np.int64)
compressed_fs.fill(-1)
y, x, _ = np.intersect1d(df.a_node.values, nlist, assume_unique=False, return_indices=True)
compressed_fs[y] = x[:]
compressed_fs[-1] = df.shape[0]
for i in range(compress_num_nodes, 1, -1):
    if compressed_fs[i - 1] == -1:
        compressed_fs[i - 1] = compressed_fs[i]

Network simplification 5.685445899999991


In [14]:
dummy_graph = deepcopy(orig_graph)

In [18]:
dummy_graph.fs = compressed_fs
dummy_graph.graph = df
dummy_graph.num_links = df.shape[0]
dummy_graph.ids = np.array(df.id.values, np.int64)
dummy_graph.set_graph('distance')
dummy_graph.set_skimming(['distance'])

In [ ]:
%%time
skm = NetworkSkimming(dummy_graph)
skm.execute()